In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

Starting by visualizing data;

In [ ]:
stock_prices = pd.read_csv('../input/daily-historical-stock-prices-1970-2018/historical_stock_prices.csv')
stocks = pd.read_csv('../input/daily-historical-stock-prices-1970-2018/historical_stocks.csv')

Let's start with a small preview of the corresponding dataframe read previously;

In [ ]:
stocks.head()

In [ ]:
stocks.columns

We realize there are 5 columns:
- 'ticker' corresponds to the name of the share
- 'exchange' corresponds to the type of exchange made 
- 'name' refers the company's name
- 'sector' refers to the actual sector where the given company operates
- 'industry' specifies the type of services that can be provided

In [ ]:
stocks.shape

In [ ]:
stocks.isnull().sum()

In [ ]:
stocks['ticker'].unique().size

We realize there are 6460 entries to the table, where the unique identifiers are the share's names, meaning that a company's name can show up twice if it has, throughout the established period of time, changed the name of it's stocks. We will note that, for having to change the type of exchange, the companies also changed the name of the shares;
The dataset presents missing values on columns 'sector' and 'industry'

In [ ]:
stocks['name'].unique().size

In [ ]:
stocks['exchange'].value_counts()

In [ ]:
stocks['exchange'].value_counts().plot(kind='bar', title='Types of exchanges')

The number of shares in each type of exchange is rather balanced, which is good for the purpose of ML.

In [ ]:
stocks['sector'].value_counts()

In [ ]:
stocks = stocks[stocks['sector'] != 'SECTOR']
stocks.shape
#remover a linha dummy

In [ ]:
stocks['sector'].value_counts().plot(kind='bar', title='Sectors')

In [ ]:
stocks['sector'].value_counts().plot(kind='pie', title='Sectors')

We realize the, throughout the dataset, the Finances sector dominates the sector column, up there with Consumer services and Health care. We can also consider technology, if we allow such leverage.

In [ ]:
stocks['industry'].value_counts()

In [ ]:
absolute_frequency_top10 = stocks['industry'].value_counts()[:10].copy()
absolute_frequency_top10 = absolute_frequency_top10.rename('')
absolute_frequency_top10.plot(kind='bar')

In [ ]:
absolute_frequency_top10.plot(kind='pie')

We can observe two major industries ruling the dataset by a considerable margin: Major Pharmaceuticals and Major Banks.

In [ ]:
industry_factor = stocks.groupby('name')['industry'].apply(lambda x: x)
sector_factor = stocks.groupby('name')['sector','industry'].agg(['unique'])
sector_factor

In [ ]:
stocks.groupby('name')['industry'].apply(lambda x: x)

In [ ]:
exchange_factor = stocks.groupby('name')['exchange'].apply(list)#.apply( lambda x: len(np.unique(x)))
ticker_factor = stocks.groupby('name')['ticker'].apply(list)
#sector_x_industry = stocks.groupby('name')['sector','industry'].agg(sector=('sector','unique'), industry=('industry','unique'))
sector_x_industry = stocks.groupby('name')['sector','industry'].agg(['unique'])
exchange_x_ticker = pd.concat([exchange_factor,ticker_factor,sector_x_industry],axis=1)
dif_exchange_x_ticker = exchange_x_ticker[exchange_x_ticker.apply(lambda x: len(x['exchange'])+len(x['ticker'])>2,axis=1)]
num_of_changes = pd.DataFrame(dif_exchange_x_ticker.apply(lambda x: len(x['exchange']),axis=1),columns=['number of changes'])
dif_exchange_x_ticker = pd.concat([dif_exchange_x_ticker,num_of_changes],axis=1)

In [ ]:
dif_exchange_x_ticker #existem missing values, trabalhar nisso

In [ ]:
top10_changes = dif_exchange_x_ticker.sort_values(by=['number of changes'],ascending=False)[:10]
top10_changes.plot(kind='bar')

Here we register the companies with most changes of names for the shares